# Ejercicio

- Vamos a hacer un algoritmo que cada 200 días rebalancee todos los activos para tener el mismo porcertaje.
- Estamos en modo backtesting: calculamos primero todas las allocations, las insertamos y ejecutamos el backtesting.

1. Descarga el maestro de valores.
2. Descarga todos los datos para cada ticker del maestro. Baja solo el close.
3. Con las series close, crea un datafame donde tengas como columnas los tickers y filas las fechas.
4. Recorre este dataframe cada 200 filas y crea una lista de allocations con valor 1/n_activos. 
5. Envía el post de estos allocations.
6. Usa la API para obtener todas las allocations introducidas.
7. Usa la API para ejecutar el backtesting.
8. Elimina todas las allocations.
9. Refactoriza el código en una clase para el algoritmo y otra para un handler del API.
10. Ejecutalo desde la terminal.